In [8]:
import os
import matplotlib.pyplot as plt
import SimpleITK as sitk

from MRLCinema.readcine.readcines import readcines
from MRLCinema.extract_motion import extract_motion
from MRLCinema.motion_statistics import motion_statistics
from MRLCinema.patient_data import read_cine_patient_ID, find_cine_frame_of_reference
from MRLCinema.patient_data import find_structure_set, find_plan, prescription
from U2Dose.dicomio.rtstruct import RtStruct
from U2Dose.dicomio.rtplan import RtPlan


ImportError: cannot import name 'prescription' from 'MRLCinema.patient_data' (/home/david/source/MRLCinema/patient_data.py)

In [ ]:
path = '/mnt/P/TERAPI/FYSIKER/David_Tilly/cine_conversion/1.3.46.670589.11.79101.5.0.17796.2024112808532524002'

patient_data_root= f'/mnt/P/TERAPI/MRLINAC/QA/RTQADATA/PATIENT_DATA'
patient_ID = read_cine_patient_ID(path)
frame_of_ref = find_cine_frame_of_reference(path)
rtss = find_structure_set(patient_data_root, patient_ID, frame_of_ref)
rtplan = find_plan(patient_data_root, patient_ID, frame_of_ref)
prescribed_dose, number_of_fractions = prescription(rtplan)

rtss.all_roi_names()

('42.7', '7')

In [3]:
cine_directory = os.path.join(path, 'TwoDImages')

import time
start = time.time()
cines = readcines(cine_directory, max_n=1500)

In [4]:
time_transversal, displacements_transversal, time_sagittal, displacements_sagittal, time_coronal, displacements_coronal = extract_motion(cines, rtss=rtss, max_n=1500) 
motion_statistics(displacements_transversal, displacements_sagittal, displacements_coronal, percentile=0.98)

image info (-77.52899867296219, -57.70499688386917, 0.0) (1.2979999780654907, 1.2979999780654907, 1.0) (124, 136, 507) 2 (1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0)
mask info (-77.52899867296219, -57.70499688386917, 0.0) (1.2979999780654907, 1.2979999780654907, 1.0) (124, 136, 507) 1 (1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0)

start parameter map
ParameterObject (0x3ddc23a0)
  RTTI typeinfo:   elastix::ParameterObject
  Reference Count: 1
  Modified Time: 355624
  Debug: Off
  Object Name: 
  Observers: 
    none
ParameterMap 0: 
  (AutomaticParameterEstimation "true")
  (CheckNumberOfSamples "true")
  (DefaultPixelValue 0)
  (FinalBSplineInterpolationOrder 3)
  (FinalGridSpacingInPhysicalUnits 8)
  (FixedImagePyramid "FixedSmoothingImagePyramid")
  (GridSpacingSchedule 2.80322 1.9881 1.41 1)
  (ImageSampler "RandomCoordinate")
  (Interpolator "ReducedDimensionBSplineInterpolator")
  (MaximumNumberOfIterations 256)
  (MaximumNumberOfSamplingAttempts 8)
  (Metric "VarianceOverLastD

In [ ]:

%matplotlib ipympl
from ipywidgets import *


x = time_sagittal
y = displacements_sagittal[:,1]
fig = plt.figure(figsize=(25/2.54, 19/2.54))
gs = fig.add_gridspec(4, 3)

ax_fixed = fig.add_subplot(gs[0, 0])
ax_moving = fig.add_subplot(gs[0, 2])

ax_x = fig.add_subplot(gs[1, 0:2])
ax_y = fig.add_subplot(gs[2, 0:2])
ax_z = fig.add_subplot(gs[3, 0:2])

#np_fixed = sitk.GetArrayFromImage(cines_sagittal_cropped[10])
#im_fixed = ax_fixed.imshow(np_fixed, cmap='gray', origin='lower', vmin=0, vmax=2048)

#np_moving = sitk.GetArrayFromImage(cines_sagittal_cropped[10])
#im_moving = ax_moving.imshow(np_moving, cmap='gray', origin='lower', vmin=0, vmax=2048)

ax_x = fig.add_subplot(gs[1, 0:2])
ax_x.plot(x, y)
ax_x.set_xlim([0, 800])
ax_x.set_ylim([-5, 5])
ax_x.grid()
ax_x.set_xlabel('Time [s]')
ax_x.set_ylabel('Displacement [mm]')

t = time_transversal
x = displacements_transversal[:,0]
ax_x.plot(t, x, label='Transversal', color='C0', marker='o', markersize=3)

t = time_coronal
x = displacements_coronal[:,0]
ax_x.plot(t, x, label='Coronal', color='C0', marker='o', markersize=3)


time_marker, = ax_x.plot([0,0],[-5, 5])

#ax_histogram = fig.add_subplot(gs[1, 2])
#ax_histogram.hist(sitk.GetArrayFromImage(cines_sagittal_cropped[10]).flatten(), bins=100, color='blue', alpha=0.5)

fig.tight_layout()

def update(w=1):

    time_marker.set_xdata([x[w],x[w]])
    ax_time_x.set_title(f"Time: {x[w]:.2f} s")

    #np_moving = sitk.GetArrayFromImage(cines_sagittal_cropped[w])
    #im_moving.set_array(np_moving)

    #ax_histogram.clear()
    #ax_histogram.hist(np_fixed.flatten(), bins=100, color='blue', alpha=0.5)
    #ax_histogram.hist(np_moving.flatten(), bins=100, color='red', alpha=0.5)

    fig.canvas.draw_idle()

interact(update, w=widgets.IntSlider(value=0, min=0, max=len(time_sagittal)-1,step=1));

NameError: name 'plt' is not defined